# Virtual Experiments

## Purpose

This notebook teaches you how to plan and execute virtual experiments with interactive experiment design. You'll learn experiment design (factorial, LHS, random, DoE), parameter range extraction, experiment execution, and result analysis with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Design virtual experiments (factorial, LHS, random, grid, DoE)
- ✅ Extract parameter ranges from warehouse data
- ✅ Execute experiments (local, VM, cloud)
- ✅ Analyze experiment results
- ✅ Compare experiments and optimize parameters
- ✅ Validate results against warehouse data

## Estimated Duration

90-120 minutes

---

## Overview

Virtual experiments enable systematic exploration of parameter spaces for process optimization. The AM-QADF framework provides comprehensive capabilities:

- 🧪 **Experiment Design**: Factorial, LHS, Random, Grid, DoE designs
- 📊 **Parameter Ranges**: Extract from warehouse or manual entry
- ⚡ **Execution**: Local, VM, or cloud execution with parallel workers
- 📈 **Analysis**: Statistical analysis, correlation, optimization
- 🔄 **Comparison**: Compare experiments and validate with warehouse data

Use the interactive widgets below to design and execute virtual experiments - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import qmc
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import virtual experiment classes
EXPERIMENT_AVAILABLE = False
try:
    from am_qadf.analytics.virtual_experiments.client import VirtualExperimentClient
    EXPERIMENT_AVAILABLE = True
    print("✅ Virtual experiment classes available")
except ImportError as e:
    print(f"⚠️ Virtual experiment classes not available: {e} - using demo mode")

# MongoDB connection setup
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None
voxel_storage = None
stl_client = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    from am_qadf.voxel_domain import VoxelGridStorage
    from am_qadf.query import STLModelClient
    
    # Initialize MongoDB connection
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    if mongo_client.is_connected():
        voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
        stl_client = STLModelClient(mongo_client=mongo_client)
        INFRASTRUCTURE_AVAILABLE = True
        print(f"✅ Connected to MongoDB: {config.database}")
    else:
        print("⚠️ MongoDB connection failed")
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env


✅ Virtual experiment classes available


Failed to connect to MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 696013662036a9fee72b2ff8, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


⚠️ MongoDB not available: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 696013662036a9fee72b2ff8, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]> - using demo mode
✅ Setup complete!


## Interactive Virtual Experiments Interface

Use the widgets below to design and execute virtual experiments. Select experiment mode, configure design, and visualize results interactively!


In [2]:
# Create Interactive Virtual Experiments Interface

# Global state
experiment_design = {}
experiment_results = {}
parameter_configs = {}
current_model_id = None
current_grid_id = None
loaded_grid_data = None
signal_arrays = {}

# ============================================
# Helper Functions for Demo Data
# ============================================

def demo_model_function(params):
    """Demo model function for virtual experiments."""
    # Simple quality model
    laser_power = params.get('laser_power', 200)
    scan_speed = params.get('scan_speed', 1.0)
    layer_thickness = params.get('layer_thickness', 0.05)
    
    # Normalize inputs
    x1 = (laser_power - 150) / 150
    x2 = (scan_speed - 0.5) / 1.5
    x3 = (layer_thickness - 0.02) / 0.08
    
    # Model output (quality score)
    quality = (
        0.4 * x1 +
        0.3 * x2 +
        0.2 * x3 +
        0.1 * x1 * x2 +
        np.random.normal(0, 0.02)  # Noise
    )
    
    return {
        'quality': max(0, min(1, quality)),
        'density': 7.8 + 0.2 * quality + np.random.normal(0, 0.05),
        'roughness': 0.5 - 0.3 * quality + np.random.normal(0, 0.05)
    }

# ============================================
# Top Panel: Data Source and Grid Selection
# ============================================

# Data source mode
data_source_label = widgets.HTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('MongoDB', 'mongodb'), ('Sample Data', 'sample')],
    value='mongodb',
    description='Source:',
    style={'description_width': 'initial'}
)

# Model selection (for MongoDB)
model_label = widgets.HTML("<b>Model:</b>")
model_options = [("━━━ Select Model ━━━", None)]
if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")

model_dropdown = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Grid type filter
grid_type_label = widgets.HTML("<b>Grid Type:</b>")
grid_type_filter = Dropdown(
    options=[
        ('All Grids', 'all'),
        ('Fused', 'fused'),
        ('Corrected', 'corrected'),
        ('Processed', 'processed'),
        ('Signal-Mapped', 'signal_mapped'),
        ('Raw', 'raw')
    ],
    value='fused',  # Default to fused grids
    description='Type:',
    style={'description_width': 'initial'}
)

# Grid selection (for MongoDB)
grid_label = widgets.HTML("<b>Grid:</b>")
grid_dropdown = Dropdown(
    options=[("━━━ Select Grid ━━━", None)],
    value=None,
    description='Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='500px')
)

load_grid_button = Button(
    description='Load Grid',
    button_style='info',
    icon='folder-open',
    layout=Layout(width='120px')
)

# Experiment mode
experiment_mode = RadioButtons(
    options=[
        ('Design', 'design'),
        ('Execute', 'execute'),
        ('Analyze', 'analyze'),
        ('Compare', 'compare')
    ],
    value='design',
    description='Mode:',
    style={'description_width': 'initial'}
)

experiment_label = widgets.HTML("<b>Experiment:</b>")
experiment_selector = Dropdown(
    options=[('New Experiment', 'new'), ('Experiment 001', 'exp001'), ('Experiment 002', 'exp002')],
    value='new',
    description='Experiment:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Experiments',
    button_style='success',
    icon='play',
    layout=Layout(width='180px')
)

analyze_button = Button(
    description='Analyze Results',
    button_style='primary',
    icon='chart-line',
    layout=Layout(width='160px')
)

top_panel = VBox([
    HBox([data_source_label, data_source_mode, experiment_mode, experiment_label, experiment_selector]),
    HBox([model_label, model_dropdown, grid_type_label, grid_type_filter]),
    HBox([grid_label, grid_dropdown, load_grid_button]),
    HBox([execute_button, analyze_button])
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Experiment Design Configuration
# ============================================

# Design Configuration Section
design_label = widgets.HTML("<b>Design Configuration:</b>")
design_type = Dropdown(
    options=[('Factorial', 'factorial'), ('LHS', 'lhs'), ('Random', 'random'), ('Grid', 'grid'), ('DoE', 'doe')],
    value='lhs',
    description='Design Type:',
    style={'description_width': 'initial'}
)
sample_size = IntSlider(value=100, min=10, max=10000, step=10, description='Sample Size:', style={'description_width': 'initial'})

design_section = VBox([
    design_label,
    design_type,
    sample_size
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Parameter Configuration (simplified - using fixed parameters)
param_config_label = widgets.HTML("<b>Parameter Configuration:</b>")

# Parameter 1: Laser Power
laser_power_min = FloatSlider(value=150, min=100, max=200, step=10, description='Laser Power Min:', style={'description_width': 'initial'})
laser_power_max = FloatSlider(value=300, min=250, max=400, step=10, description='Laser Power Max:', style={'description_width': 'initial'})
laser_power_levels = IntSlider(value=5, min=2, max=20, step=1, description='Levels:', style={'description_width': 'initial'})

laser_power_config = VBox([
    widgets.HTML("<b>Laser Power:</b>"),
    laser_power_min,
    laser_power_max,
    laser_power_levels
], layout=Layout(padding='5px'))

# Parameter 2: Scan Speed
scan_speed_min = FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='Scan Speed Min:', style={'description_width': 'initial'})
scan_speed_max = FloatSlider(value=2.0, min=1.5, max=3.0, step=0.1, description='Scan Speed Max:', style={'description_width': 'initial'})
scan_speed_levels = IntSlider(value=5, min=2, max=20, step=1, description='Levels:', style={'description_width': 'initial'})

scan_speed_config = VBox([
    widgets.HTML("<b>Scan Speed:</b>"),
    scan_speed_min,
    scan_speed_max,
    scan_speed_levels
], layout=Layout(padding='5px'))

# Parameter 3: Layer Thickness
layer_thickness_min = FloatSlider(value=0.02, min=0.01, max=0.05, step=0.01, description='Layer Thickness Min:', style={'description_width': 'initial'})
layer_thickness_max = FloatSlider(value=0.1, min=0.08, max=0.15, step=0.01, description='Layer Thickness Max:', style={'description_width': 'initial'})
layer_thickness_levels = IntSlider(value=5, min=2, max=20, step=1, description='Levels:', style={'description_width': 'initial'})

layer_thickness_config = VBox([
    widgets.HTML("<b>Layer Thickness:</b>"),
    layer_thickness_min,
    layer_thickness_max,
    layer_thickness_levels
], layout=Layout(padding='5px'))

# Parameter accordion
param_accordion = Accordion(children=[
    laser_power_config,
    scan_speed_config,
    layer_thickness_config
])
param_accordion.set_title(0, 'Laser Power')
param_accordion.set_title(1, 'Scan Speed')
param_accordion.set_title(2, 'Layer Thickness')

param_section = VBox([
    param_config_label,
    param_accordion
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Parameter Ranges Section
ranges_label = widgets.HTML("<b>Parameter Ranges:</b>")
load_from_warehouse = Checkbox(value=False, description='Load from Warehouse', style={'description_width': 'initial'})
warehouse_source = Dropdown(
    options=[('Build 001', 'build001'), ('Build 002', 'build002'), ('Build 003', 'build003')],
    value='build001',
    description='Source:',
    style={'description_width': 'initial'}
)

ranges_section = VBox([
    ranges_label,
    load_from_warehouse,
    warehouse_source
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Execution Configuration Section
exec_label = widgets.HTML("<b>Execution Configuration:</b>")
execution_mode = RadioButtons(
    options=[('Local', 'local'), ('VM', 'vm'), ('Cloud', 'cloud')],
    value='local',
    description='Mode:',
    style={'description_width': 'initial'}
)
parallel_workers = IntSlider(value=4, min=1, max=100, step=1, description='Workers:', style={'description_width': 'initial'})
timeout = IntSlider(value=3600, min=1, max=3600, step=60, description='Timeout (s):', style={'description_width': 'initial'})

exec_section = VBox([
    exec_label,
    execution_mode,
    parallel_workers,
    timeout
], layout=Layout(padding='5px', border='1px solid #ddd'))

left_panel = VBox([
    design_section,
    param_section,
    ranges_section,
    exec_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Helper Functions for MongoDB
# ============================================

def update_grid_dropdown(change=None):
    """Update grid dropdown when model or grid type changes."""
    global current_model_id
    
    model_id = model_dropdown.value
    grid_type = grid_type_filter.value
    
    if not model_id:
        grid_dropdown.options = [("━━━ Select Grid ━━━", None)]
        return
    
    current_model_id = model_id
    
    if not voxel_storage:
        grid_dropdown.options = [("━━━ MongoDB not available ━━━", None)]
        return
    
    try:
        # Get all grids for this model
        grids = voxel_storage.list_grids(model_id=model_id, limit=100)
        
        grid_options = [("━━━ Select Grid ━━━", None)]
        for grid in grids:
            metadata = grid.get('metadata', {})
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # Determine grid type
            is_fused = config_meta.get('fusion_applied', False)
            is_corrected = config_meta.get('correction_applied', False)
            is_processed = config_meta.get('processing_applied', False)
            has_signals = len(grid.get('available_signals', [])) > 0
            
            grid_type_match = False
            if grid_type == 'all':
                grid_type_match = True
            elif grid_type == 'fused' and is_fused:
                grid_type_match = True
            elif grid_type == 'corrected' and is_corrected:
                grid_type_match = True
            elif grid_type == 'processed' and is_processed:
                grid_type_match = True
            elif grid_type == 'signal_mapped' and has_signals and not is_corrected and not is_processed and not is_fused:
                grid_type_match = True
            elif grid_type == 'raw' and not has_signals:
                grid_type_match = True
            
            if grid_type_match:
                grid_id = grid.get('grid_id', str(grid.get('_id', '')))
                grid_name = grid.get('grid_name', 'Unknown')
                n_signals = len(grid.get('available_signals', []))
                
                # Build status label
                status_parts = []
                if is_fused:
                    status_parts.append('fused')
                if is_corrected:
                    status_parts.append('corrected')
                if is_processed:
                    status_parts.append('processed')
                if has_signals and not status_parts:
                    status_parts.append('mapped')
                if not status_parts:
                    status_parts.append('raw')
                
                status_str = ', '.join(status_parts)
                label = f"{grid_name} ({n_signals} signal(s), {status_str}) ({grid_id[:8]}...)"
                grid_options.append((label, grid_id))
        
        if len(grid_options) == 1:
            grid_options.append(("No grids found matching filter", None))
        
        grid_dropdown.options = grid_options
    except Exception as e:
        grid_dropdown.options = [("━━━ Error loading grids ━━━", None)]
        print(f"⚠️ Error loading grids: {e}")

def load_grid_from_mongodb(button):
    """Load selected grid from MongoDB."""
    global current_model_id, current_grid_id, loaded_grid_data, signal_arrays
    
    if not voxel_storage or not grid_dropdown.value:
        status_display.value = "<b>Status:</b> <span style='color: red;'>⚠️ Please select a grid to load</span>"
        return
    
    grid_id = grid_dropdown.value
    current_grid_id = grid_id
    
    status_display.value = "<b>Status:</b> Loading grid from MongoDB..."
    progress_bar.value = 0
    
    try:
        # Load grid from MongoDB
        grid_data = voxel_storage.load_voxel_grid(grid_id=grid_id)
        
        if not grid_data:
            status_display.value = "<b>Status:</b> <span style='color: red;'>⚠️ Failed to load grid</span>"
            return
        
        # Extract data from dictionary
        signal_arrays = grid_data.get('signal_arrays', {})
        metadata = grid_data.get('metadata', {})
        grid_name = grid_data.get('grid_name', 'Unknown')
        
        # Store loaded data
        loaded_grid_data = {
            'grid_data': grid_data,
            'metadata': metadata,
            'signal_arrays': signal_arrays
        }
        
        # Extract parameter ranges from metadata if available
        config_meta = metadata.get('configuration_metadata', {})
        if not config_meta:
            config_meta = metadata
        
        # Update parameter range sliders if process parameters are available
        if config_meta.get('process_parameters'):
            process_params = config_meta.get('process_parameters', {})
            # Try to extract ranges from process parameters
            for param_name, param_value in process_params.items():
                if isinstance(param_value, (int, float)):
                    # Set slider ranges based on parameter value (with some margin)
                    if param_name == 'laser_power' or 'laser_power' in param_name.lower():
                        laser_power_min.value = max(0, param_value * 0.5)
                        laser_power_max.value = param_value * 2.0
                    elif param_name == 'scan_speed' or 'scan_speed' in param_name.lower():
                        scan_speed_min.value = max(0.1, param_value * 0.5)
                        scan_speed_max.value = param_value * 2.0
                    elif param_name == 'layer_thickness' or 'layer_thickness' in param_name.lower():
                        layer_thickness_min.value = max(0.01, param_value * 0.5)
                        layer_thickness_max.value = param_value * 2.0
        
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Loaded grid: {grid_name} ({len(signal_arrays)} signal(s))</span>"
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>❌ Error loading grid: {str(e)}</span>"
        progress_bar.value = 0
        import traceback
        traceback.print_exc()

# Function to update UI based on data source mode
def update_data_source_mode(change):
    """Show/hide MongoDB widgets based on data source mode."""
    if change['new'] == 'mongodb':
        model_dropdown.layout.display = 'flex'
        grid_type_filter.layout.display = 'flex'
        grid_dropdown.layout.display = 'flex'
        load_grid_button.layout.display = 'flex'
    else:
        model_dropdown.layout.display = 'none'
        grid_type_filter.layout.display = 'none'
        grid_dropdown.layout.display = 'none'
        load_grid_button.layout.display = 'none'

# Connect events
data_source_mode.observe(update_data_source_mode, names='value')
update_data_source_mode({'new': data_source_mode.value})
model_dropdown.observe(update_grid_dropdown, names='value')
grid_type_filter.observe(update_grid_dropdown, names='value')
load_grid_button.on_click(load_grid_from_mongodb)

# ============================================
# Center Panel: Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Design', 'design'), ('Results', 'results'), ('Analysis', 'analysis'), ('Comparison', 'comparison')],
    value='design',
    description='View:',
    style={'description_width': 'initial'}
)

viz_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Experiment Visualization</h3>"),
    viz_mode,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Results
# ============================================

# Design Statistics
design_stats_label = widgets.HTML("<b>Design Statistics:</b>")
design_stats_display = widgets.HTML("No design created yet")
design_stats_section = VBox([
    design_stats_label,
    design_stats_display
], layout=Layout(padding='5px'))

# Execution Status
exec_status_label = widgets.HTML("<b>Execution Status:</b>")
exec_status_display = widgets.HTML("No execution yet")
exec_status_section = VBox([
    exec_status_label,
    exec_status_display
], layout=Layout(padding='5px'))

# Results Summary
results_summary_label = widgets.HTML("<b>Results Summary:</b>")
results_summary_display = widgets.HTML("No results available")
results_summary_section = VBox([
    results_summary_label,
    results_summary_display
], layout=Layout(padding='5px'))

# Analysis Results
analysis_results_label = widgets.HTML("<b>Analysis Results:</b>")
analysis_results_display = widgets.HTML("No analysis available")
analysis_results_section = VBox([
    analysis_results_label,
    analysis_results_display
], layout=Layout(padding='5px'))

# Comparison Results
comparison_label = widgets.HTML("<b>Comparison:</b>")
comparison_display = widgets.HTML("No comparison available")
comparison_section = VBox([
    comparison_label,
    comparison_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_design_button = Button(description='Export Design', button_style='', layout=Layout(width='150px'))
export_results_button = Button(description='Export Results', button_style='', layout=Layout(width='150px'))
export_analysis_button = Button(description='Export Analysis', button_style='', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_design_button,
    export_results_button,
    export_analysis_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    design_stats_section,
    exec_status_section,
    results_summary_section,
    analysis_results_section,
    comparison_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to design experiments")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
info_display = widgets.HTML("")

bottom_panel = VBox([
    status_display,
    progress_bar,
    info_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Experiment Functions
# ============================================

def generate_design_points():
    """Generate design points based on configuration."""
    global experiment_design
    
    # Parameter ranges
    param_ranges = {
        'laser_power': (laser_power_min.value, laser_power_max.value),
        'scan_speed': (scan_speed_min.value, scan_speed_max.value),
        'layer_thickness': (layer_thickness_min.value, layer_thickness_max.value)
    }
    
    n_samples = sample_size.value
    design_type_val = design_type.value
    
    design_points = []
    
    if design_type_val == 'lhs':
        # Latin Hypercube Sampling
        try:
            sampler = qmc.LatinHypercube(d=len(param_ranges))
            samples = sampler.random(n=n_samples)
            
            param_names = list(param_ranges.keys())
            for sample in samples:
                point = {}
                for i, name in enumerate(param_names):
                    min_val, max_val = param_ranges[name]
                    point[name] = min_val + sample[i] * (max_val - min_val)
                design_points.append(point)
        except:
            # Fallback to random
            for _ in range(n_samples):
                point = {}
                for name, (min_val, max_val) in param_ranges.items():
                    point[name] = np.random.uniform(min_val, max_val)
                design_points.append(point)
    
    elif design_type_val == 'factorial':
        # Factorial design (simplified)
        levels = [laser_power_levels.value, scan_speed_levels.value, layer_thickness_levels.value]
        param_names = list(param_ranges.keys())
        
        for i in range(min(levels)):
            point = {}
            for j, name in enumerate(param_names):
                min_val, max_val = param_ranges[name]
                # Create factorial levels
                level_val = min_val + (max_val - min_val) * (i / (levels[j] - 1)) if levels[j] > 1 else min_val
                point[name] = level_val
            design_points.append(point)
    
    elif design_type_val == 'random':
        # Random sampling
        for _ in range(n_samples):
            point = {}
            for name, (min_val, max_val) in param_ranges.items():
                point[name] = np.random.uniform(min_val, max_val)
            design_points.append(point)
    
    elif design_type_val == 'grid':
        # Grid design (simplified)
        levels = [laser_power_levels.value, scan_speed_levels.value, layer_thickness_levels.value]
        param_names = list(param_ranges.keys())
        
        for i in range(levels[0]):
            for j in range(levels[1]):
                for k in range(levels[2]):
                    point = {}
                    for idx, name in enumerate(param_names):
                        min_val, max_val = param_ranges[name]
                        level_idx = [i, j, k][idx]
                        level_val = min_val + (max_val - min_val) * (level_idx / (levels[idx] - 1)) if levels[idx] > 1 else min_val
                        point[name] = level_val
                    design_points.append(point)
    
    else:  # doe
        # DoE (simplified - use LHS)
        sampler = qmc.LatinHypercube(d=len(param_ranges))
        samples = sampler.random(n=n_samples)
        param_names = list(param_ranges.keys())
        for sample in samples:
            point = {}
            for i, name in enumerate(param_names):
                min_val, max_val = param_ranges[name]
                point[name] = min_val + sample[i] * (max_val - min_val)
            design_points.append(point)
    
    experiment_design = {
        'design_type': design_type_val,
        'n_points': len(design_points),
        'design_points': design_points,
        'param_ranges': param_ranges
    }
    
    return design_points

def execute_experiments(button):
    """Execute virtual experiments."""
    global experiment_results, experiment_design
    
    status_display.value = "<b>Status:</b> Executing experiments..."
    progress_bar.value = 0
    info_display.value = ""
    
    try:
        # Generate design if not exists
        if not experiment_design:
            design_points = generate_design_points()
        else:
            design_points = experiment_design['design_points']
        
        progress_bar.value = 20
        
        # Execute experiments (simplified - evaluate model function)
        results = []
        n_points = len(design_points)
        
        # Use real model function if MongoDB data is loaded, otherwise use demo
        model_function = demo_model_function
        if data_source_mode.value == 'mongodb' and loaded_grid_data and signal_arrays:
            # Create a model function based on loaded grid data
            # This is a simplified version - in practice, you'd use the actual model
            def real_model_function(params):
                # Extract signal values or quality metrics based on parameters
                # This is a placeholder - actual implementation would use the model
                quality = 0.5  # Default
                if signal_arrays:
                    # Use mean signal values as quality proxy
                    mean_signals = {name: np.mean(arr.flatten()) for name, arr in signal_arrays.items()}
                    # Simple quality calculation (can be enhanced)
                    quality = np.mean(list(mean_signals.values())) / 100.0 if mean_signals else 0.5
                
                return {
                    'quality': max(0, min(1, quality)),
                    'density': 7.8 + 0.2 * quality,
                    'roughness': 0.5 - 0.3 * quality
                }
            model_function = real_model_function
        
        for idx, point in enumerate(design_points):
            result = model_function(point)
            result['parameters'] = point
            results.append(result)
            progress_bar.value = 20 + int(70 * (idx + 1) / n_points)
        
        experiment_results = {
            'results': results,
            'n_completed': len(results),
            'n_failed': 0,
            'execution_time': n_points * 0.1  # Simulated
        }
        
        progress_bar.value = 90
        
        # Update displays
        update_results_display()
        update_visualization()
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Experiments completed</span>"
        info_display.value = f"<p>Completed: <b>{len(results)}</b> experiments | Time: <b>{experiment_results['execution_time']:.1f}</b> s</p>"
        
    except Exception as e:
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during execution</span>"
        progress_bar.value = 0

def analyze_results(button):
    """Analyze experiment results."""
    global experiment_results
    
    if not experiment_results or 'results' not in experiment_results:
        info_display.value = "<span style='color: orange;'>⚠️ Please execute experiments first</span>"
        return
    
    status_display.value = "<b>Status:</b> Analyzing results..."
    progress_bar.value = 0
    
    try:
        results = experiment_results['results']
        progress_bar.value = 50
        
        # Extract quality values
        quality_values = [r['quality'] for r in results]
        
        # Statistical analysis
        analysis = {
            'mean_quality': np.mean(quality_values),
            'std_quality': np.std(quality_values),
            'min_quality': np.min(quality_values),
            'max_quality': np.max(quality_values),
            'best_result': results[np.argmax(quality_values)],
            'worst_result': results[np.argmin(quality_values)]
        }
        
        experiment_results['analysis'] = analysis
        progress_bar.value = 80
        
        # Update displays
        update_analysis_display()
        update_visualization()
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Analysis completed</span>"
        
    except Exception as e:
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during analysis</span>"
        progress_bar.value = 0

def update_results_display():
    """Update results displays."""
    global experiment_design, experiment_results
    
    # Design statistics
    if experiment_design:
        stats_html = f"<p><b>Design Points:</b> {experiment_design['n_points']:,}</p>"
        stats_html += f"<p><b>Design Type:</b> {experiment_design['design_type']}</p>"
        stats_html += f"<p><b>Coverage:</b> Good</p>"
        design_stats_display.value = stats_html
    
    # Execution status
    if experiment_results and 'n_completed' in experiment_results:
        exec_html = f"<p><b>Completed:</b> {experiment_results['n_completed']}</p>"
        exec_html += f"<p><b>Failed:</b> {experiment_results.get('n_failed', 0)}</p>"
        exec_html += f"<p><b>Time:</b> {experiment_results.get('execution_time', 0):.1f} s</p>"
        exec_status_display.value = exec_html
    
    # Results summary
    if experiment_results and 'results' in experiment_results:
        results = experiment_results['results']
        quality_values = [r['quality'] for r in results]
        
        summary_html = f"<p><b>Best:</b> {np.max(quality_values):.4f}</p>"
        summary_html += f"<p><b>Worst:</b> {np.min(quality_values):.4f}</p>"
        summary_html += f"<p><b>Mean:</b> {np.mean(quality_values):.4f}</p>"
        summary_html += f"<p><b>Std:</b> {np.std(quality_values):.4f}</p>"
        results_summary_display.value = summary_html

def update_analysis_display():
    """Update analysis results display."""
    global experiment_results
    
    if experiment_results and 'analysis' in experiment_results:
        analysis = experiment_results['analysis']
        analysis_html = f"<p><b>Mean Quality:</b> {analysis['mean_quality']:.4f}</p>"
        analysis_html += f"<p><b>Std Quality:</b> {analysis['std_quality']:.4f}</p>"
        analysis_html += f"<p><b>Best Quality:</b> {analysis['max_quality']:.4f}</p>"
        best_params = analysis['best_result']['parameters']
        analysis_html += f"<p><b>Best Params:</b><br>"
        for param, value in best_params.items():
            analysis_html += f"  • {param}: {value:.4f}<br>"
        analysis_html += "</p>"
        analysis_results_display.value = analysis_html

def update_visualization():
    """Update visualization display."""
    global experiment_design, experiment_results
    
    with viz_output:
        clear_output(wait=True)
        
        mode = experiment_mode.value
        viz = viz_mode.value
        
        if viz == 'design':
            if experiment_design and 'design_points' in experiment_design and experiment_design['design_points']:
                design_points = experiment_design['design_points']
                
                # Check if design points have the expected parameters
                if design_points and len(design_points) > 0:
                    # Get parameter names from first point
                    param_names = list(design_points[0].keys())
                    
                    if len(param_names) >= 2:
                        # 2D parameter space visualization
                        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                        
                        # First two parameters
                        param1_vals = [p.get(param_names[0], 0) for p in design_points]
                        param2_vals = [p.get(param_names[1], 0) for p in design_points]
                        axes[0].scatter(param1_vals, param2_vals, alpha=0.6, s=30)
                        axes[0].set_xlabel(param_names[0].replace('_', ' ').title())
                        axes[0].set_ylabel(param_names[1].replace('_', ' ').title())
                        axes[0].set_title(f'Design Points: {param_names[0]} vs {param_names[1]}')
                        axes[0].grid(True, alpha=0.3)
                        
                        # Second and third parameters (or first and third if only 2 params)
                        if len(param_names) >= 3:
                            param3_vals = [p.get(param_names[2], 0) for p in design_points]
                            axes[1].scatter(param2_vals, param3_vals, alpha=0.6, s=30)
                            axes[1].set_xlabel(param_names[1].replace('_', ' ').title())
                            axes[1].set_ylabel(param_names[2].replace('_', ' ').title())
                            axes[1].set_title(f'Design Points: {param_names[1]} vs {param_names[2]}')
                        else:
                            # If only 2 parameters, show same plot twice or different view
                            axes[1].scatter(param1_vals, param2_vals, alpha=0.6, s=30, color='green')
                            axes[1].set_xlabel(param_names[0].replace('_', ' ').title())
                            axes[1].set_ylabel(param_names[1].replace('_', ' ').title())
                            axes[1].set_title(f'Design Points: {param_names[0]} vs {param_names[1]} (View 2)')
                        axes[1].grid(True, alpha=0.3)
                        
                        plt.tight_layout()
                        plt.show()
                    else:
                        display(HTML("<p>Design points need at least 2 parameters for visualization</p>"))
                else:
                    display(HTML("<p>No design points available. Generate design first.</p>"))
            else:
                display(HTML("<p>Generate design to see visualization</p>"))
        
        elif viz == 'results':
            if experiment_results and 'results' in experiment_results and experiment_results['results']:
                results = experiment_results['results']
                
                if not results:
                    display(HTML("<p>No results available. Execute experiments first.</p>"))
                else:
                    # Get parameter names from first result
                    first_result = results[0]
                    if 'parameters' in first_result and 'quality' in first_result:
                        param_names = list(first_result['parameters'].keys())
                        
                        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                        
                        # Quality distribution
                        quality_values = [r.get('quality', 0) for r in results]
                        axes[0, 0].hist(quality_values, bins=min(30, len(quality_values)), alpha=0.7, edgecolor='black')
                        axes[0, 0].set_xlabel('Quality')
                        axes[0, 0].set_ylabel('Frequency')
                        axes[0, 0].set_title('Quality Distribution')
                        axes[0, 0].grid(True, alpha=0.3, axis='y')
                        
                        # Parameter-response plots
                        colors = ['blue', 'green', 'orange', 'red', 'purple']
                        for idx, param_name in enumerate(param_names[:4]):  # Show up to 4 parameters
                            row = idx // 2
                            col = (idx % 2) + (1 if idx >= 2 else 0)
                            if row < 2 and col < 2:
                                param_vals = [r['parameters'].get(param_name, 0) for r in results]
                                axes[row, col].scatter(param_vals, quality_values, alpha=0.6, color=colors[idx % len(colors)])
                                axes[row, col].set_xlabel(param_name.replace('_', ' ').title())
                                axes[row, col].set_ylabel('Quality')
                                axes[row, col].set_title(f'{param_name.replace("_", " ").title()} vs Quality')
                                axes[row, col].grid(True, alpha=0.3)
                        
                        # Hide unused subplots
                        for idx in range(len(param_names), 4):
                            row = idx // 2
                            col = (idx % 2) + (1 if idx >= 2 else 0)
                            if row < 2 and col < 2:
                                axes[row, col].axis('off')
                        
                        plt.tight_layout()
                        plt.show()
                    else:
                        display(HTML("<p>Results format is invalid. Expected 'parameters' and 'quality' keys.</p>"))
            else:
                display(HTML("<p>No results available. Execute experiments first.</p>"))
        
        elif viz == 'analysis' and experiment_results and 'analysis' in experiment_results:
            analysis = experiment_results['analysis']
            results = experiment_results['results']
            
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Quality statistics
            quality_values = [r['quality'] for r in results]
            axes[0].boxplot([quality_values], vert=True)
            axes[0].set_ylabel('Quality')
            axes[0].set_title('Quality Statistics')
            axes[0].grid(True, alpha=0.3, axis='y')
            
            # Best vs Worst comparison
            best_params = analysis['best_result']['parameters']
            worst_params = analysis['worst_result']['parameters']
            
            param_names = list(best_params.keys())
            best_vals = [best_params[p] for p in param_names]
            worst_vals = [worst_params[p] for p in param_names]
            
            x = np.arange(len(param_names))
            width = 0.35
            axes[1].bar(x - width/2, best_vals, width, label='Best', alpha=0.8)
            axes[1].bar(x + width/2, worst_vals, width, label='Worst', alpha=0.8)
            axes[1].set_xlabel('Parameters')
            axes[1].set_ylabel('Parameter Value')
            axes[1].set_title('Best vs Worst Parameters')
            axes[1].set_xticks(x)
            axes[1].set_xticklabels(param_names, rotation=45)
            axes[1].legend()
            axes[1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'comparison':
            if experiment_results and 'results' in experiment_results:
                display(HTML("<p>Comparison view - compare with warehouse data or other experiments</p>"))
            else:
                display(HTML("<p>Execute experiments first to enable comparison</p>"))
        else:
            # No data available
            if not experiment_design and not experiment_results:
                display(HTML("<p>Generate design and execute experiments to see visualizations</p>"))
            elif not experiment_results:
                display(HTML("<p>Execute experiments to see results visualization</p>"))

# Connect events
execute_button.on_click(execute_experiments)
analyze_button.on_click(analyze_results)
viz_mode.observe(lambda x: update_visualization(), names='value')
experiment_mode.observe(lambda x: update_visualization(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to design and execute virtual experiments.

### Key Takeaways

1. **Experiment Design**: Factorial, LHS, Random, Grid, and DoE designs
2. **Parameter Configuration**: Configure parameter ranges and levels
3. **Execution**: Local, VM, or cloud execution with parallel workers
4. **Result Analysis**: Statistical analysis, parameter-response plots, best/worst identification
5. **Visualization**: Design space visualization, results plots, analysis charts
6. **Comparison**: Compare experiments and validate with warehouse data

### Next Steps

Proceed to:
- **13_Anomaly_Detection_Methods.ipynb** - Learn anomaly detection techniques
- **14_Anomaly_Detection_Advanced.ipynb** - Learn advanced anomaly detection

### Related Resources

- Virtual Experiments Documentation: `../docs/AM_QADF/05-modules/analytics.md`
- API Reference: `../docs/AM_QADF/06-api-reference/analytics-api.md`
- Examples: `../examples/`
